# Collab preliminaries

In [ ]:
!nvcc --version

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls -l /content/gdrive/MyDrive/Wav2Lip

# Get the code and models

In [ ]:
%cd /content
!git clone "https://github.com/yangningbo/Wav2Lip.git" "wav2lip_288x288"

In [ ]:
!cp -ri "/content/gdrive/MyDrive/Wav2Lip/data.zip" "/content/wav2lip_288x288/"
!cp -ri "/content/gdrive/MyDrive/Wav2Lip/filelists.zip" "/content/wav2lip_288x288/"
!cp -ri "/content/gdrive/MyDrive/Wav2Lip/wav2lip_gan.pth" "/content/wav2lip_288x288/checkpoints/"
!cp -ri "/content/gdrive/MyDrive/Wav2Lip/s3fd.pth" "/content/wav2lip_288x288/face_detection/detection/sfd/"
!cp -ri "/content/gdrive/MyDrive/Wav2Lip/test2.mp4" "/content/gdrive/MyDrive/Wav2Lip/test.mp3" "/content/sample_data/"

# Get the pre-requisites

In [ ]:
%cd /content/wav2lip_288x288
!pip uninstall -y tensorflow tensorflow-gpu
!pip install -r requirements.txt
!pip install -q youtube-dl
!pip install -q yt-dlp
!pip install -q ffmpeg-python

In [ ]:
%cd /content/wav2lip_288x288
!unzip data.zip
!rm -rf data.zip

In [ ]:
%cd /content/wav2lip_288x288
!unzip filelists.zip
!rm -rf filelists.zip

## Pre-process data

In [ ]:
%cd /content/wav2lip_288x288
!rm -rf processed_data
!python preprocess.py --data_root '/content/wav2lip_288x288/data' --preprocessed_root '/content/wav2lip_288x288/processed_data'

## Train the Discriminator

In [ ]:
%cd /content/wav2lip_288x288
!mkdir expert_checkpoints
!python color_syncnet_train.py --data_root '/content/wav2lip_288x288/processed_data' --checkpoint_dir '/content/wav2lip_288x288/expert_checkpoints'

## Train the Generator

In [ ]:
%cd /content/wav2lip_288x288
!python wav2lip_train.py --data_root "/content/wav2lip_288x288/processed_data" --checkpoint_dir '/content/wav2lip_288x288/checkpoints' --syncnet_checkpoint_path "/content/wav2lip_288x288/expert_checkpoints/checkpoint_step000000001.pth"

# Now lets try!

In [ ]:
%cd /content/wav2lip_288x288
!python inference.py --checkpoint_path "/content/wav2lip_288x288/checkpoints/checkpoint_step000000001.pth" --face "/content/sample_data/test2.mp4" --audio "/content/sample_data/test.mp3"

## **Variations to try**

In [ ]:
# 1.   Use more padding to include the chin region
%cd /content/wav2lip_288x288
!python inference.py --checkpoint_path "/content/wav2lip_288x288/checkpoints/checkpoint_step000000001.pth" --face "/content/sample_data/test2.mp4" --audio "/content/sample_data/test.mp3" --pads 0 20 0 0

In [ ]:
# 2.   Use resize_factor to reduce the video resolution, as there is a change you might get better results for lower resolution videos. Why? Because the model was trained on low resolution faces.
%cd /content/wav2lip_288x288
!python inference.py --checkpoint_path "/content/wav2lip_288x288/checkpoints/checkpoint_step000000001.pth" --face "/content/sample_data/test2.mp4" --audio "/content/sample_data/test.mp3" --resize_factor 2

## ------------ Split line ------------

In [ ]:
!pip install yt-dlp
from urllib import parse as urlparse
YOUTUBE_URL = 'https://www.youtube.com/watch?v=vAnWYLTdvfY'
url_data = urlparse.urlparse(YOUTUBE_URL)
query = urlparse.parse_qs(url_data.query)
YOUTUBE_ID = query["v"][0]

start = 35
end = 41
interval = end - start

!yt-dlp -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID
!ffmpeg -y -i youtube.mp4 -ss {start} -t {interval} -async 1 /content/sample_data/test.mp4

In [ ]:
# Preview the trimmed video
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/sample_data/test.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""""")